In [1]:
%load_ext rpy2.ipython

In [2]:
%R require(ggplot2); require(tidyr); require(minpack.lm)

/Users/faruqia/anaconda3/envs/ppm-r-env/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)
/Users/faruqia/anaconda3/envs/ppm-r-env/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: tidyr

  warnings.warn(x, RRuntimeWarning)
/Users/faruqia/anaconda3/envs/ppm-r-env/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: minpack.lm

  warnings.warn(x, RRuntimeWarning)


array([1], dtype=int32)

In [3]:
import itertools
import numpy as np
import pandas as pd
from scipy import special, optimize
from joblib import Parallel, delayed
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from collections import Counter
from datetime import datetime

/Users/faruqia/anaconda3/envs/ppm-r-env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Define variable (i.e., column) names

In [ ]:
#################### Variables to define ###############

SALES_L1 = 'KADCYLA_SALES_SA_L1'
CALLS = 'KADCYLA_CALLS'
SALES_SA = 'KADCYLA_SALES_SA'
SPEAKER_PROG = 'KADCYLA_SP'
COPAY = 'KADCYLA_COPAY_COUNT'
EMAILS = 'KADCYLA_EMAILS'
ALERTS = 'KADCYLA_ALERTS'
WEBVISITS = 'KADCYLA_WEB_VISITS'
ZIPVAR = 'ZIP3'


#################### End of variable list ###############

# Reading data and pre-processing

In [4]:
zip3_kadcyla_s1 = pd.read_sas('../MMix_Model/Tables/zip3_kadcyla_s1.sas7bdat', format='sas7bdat',
                    index=None, iterator=False, encoding = 'latin-1')
# Add lag variable for calls
zip3_kadcyla_s1['KADCYLA_CALLS_L1'] = zip3_kadcyla_s1[CALLS].shift()
zip3_kadcyla_s1['KADCYLA_SP_L1'] = zip3_kadcyla_s1[SPEAKER_PROG].shift()


# Fill NAs with zeros
zip3_kadcyla_s1 = zip3_kadcyla_s1.fillna(0)
zip3_kadcyla_s1[list(zip3_kadcyla_s1.columns[2:])] = zip3_kadcyla_s1[list(zip3_kadcyla_s1.columns[2:])].apply(pd.to_numeric)

# Set negative values to zero for sales. DDD data 
zip3_kadcyla_s1[SALES_SA] = zip3_kadcyla_s1[SALES_SA].clip(lower=0)

# Scipy implementation

In [5]:
# Check documentation here
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

# NOTE: ADD ARGUMENT BACK IF TESTING DOESN'T WORK
def model1(params,x1,x2,x4): #,x3
    """
    params = [b0,b1,b2, r2]  #,r1,
    x1, x2, x4 = SALES_SA_L1, CALLS, SALES_SA  #x3 = CALLS_L1,
    """
    return params[0] + params[1] * x1 + params[2] * (1 - np.exp(-params[3] * 
            x2)) - x4
    # NOTE: UNCOMMENT IF TESTING DOESN'T WORK
    #return params[0] + params[1] * x1 + (1 - np.exp(-params[3] * 
    #        x2)) + params[2] * (1 - np.exp(-params[4] * 
    #        x3)) - x4

def model2(params,x1,x2,x3):
    """
    params = [b3,r3,b4,r4]
    x1, x2, x3 = SP, COPAY_COUNT, residual_a
    """
    return  params[0] * (1 - np.exp(-params[1] * x1)) + params[2] * (1 - np.exp(-params[3] * 
    x2)) - x3


def model3(params,x1,x2,x3,x4):
    """
    params = [b5,r5,b6,r6,b7,r7]
    x1,x2,x3,x4 = EMAILS, ALERTS, WEB_VISITS, residual_b
    """
    return params[0] * (1 - np.exp(-params[1] * x1)) + params[2] * (1 - np.exp(-params[3] * 
        x2)) + params[4] * (1 - np.exp(-params[5] * 
        x3)) - x4


#NOTE: ADD BACK IF NEEDED
#callsl1,
def model_optimzer(datainput,zipcode,salesl1,calls,
                  salessa,sp,copay,
                  emails,alerts,webvisits):
    data_sub = datainput[datainput['ZIP3'] == zipcode]
    #NOTE: ADD PARAMETER BACK IF TESTING DOESN'T WORK
    parameters1 = [0,0,0,0.01] #0.01,
    result1 = optimize.least_squares(model1,parameters1,args=(
    data_sub[salesl1],
    data_sub[calls],
    #data_sub[callsl1],
    data_sub[salessa]
    ))
    # result1.x gives the values of the estimated parameters
    # Gives the residual of the model1
    residual_a = result1.fun
    parameters2 = [0,0.01,0,0.01]
    result2 = optimize.least_squares(model2,parameters2,args=(
        data_sub[sp],
        data_sub[copay],
        residual_a
    ))
    residual_b = result2.fun
    parameters3 = [0,0.01,0,0.01,0,0.01]
    result3 = optimize.least_squares(model3,parameters3,args=(
        data_sub[emails],
        data_sub[alerts],
        data_sub[webvisits],
        residual_b
    ))
    output_dict = dict()
    output_dict[zipcode] = [result1,result2,result3]
    return output_dict



# Start datetime
startTime = datetime.now()

unique_zips = list(zip3_kadcyla_s1[ZIPVAR].unique())

    
# Running the model
#https://zacharyst.com/2016/03/31/parallelize-a-multifunction-argument-in-python/
results =  Parallel(n_jobs=4)(delayed(model_optimzer)(datainput=zip3_kadcyla_s1,zipcode=zipc,
            salesl1=SALES_L1,
calls=CALLS,salessa=SALES_SA,
sp=SPEAKER_PROG,copay=COPAY,emails=EMAILS,
alerts=ALERTS,webvisits=WEBVISITS) for zipc in unique_zips)
#NOTE: ADD BACK IF NEEDED
#callsl1='KADCYLA_CALLS_L1'


print (str(datetime.now()-startTime))



# Without joblib
#0:29:22.997824

#With joblib
#0:15:00.261307

0:11:47.359589


In [6]:
zip_counts = dict(Counter(zip3_kadcyla_s1['ZIP3']))

b0_df = [] 
b1_df = []
b2_df = []
# NOTE: UNCOMMENT r1_df AFTER TESTING
#r1_df = []
r2_df = []
b3_df = []
r3_df = []
b4_df = []
r4_df = []
b5_df = []
r5_df = []
b6_df = []
r6_df = []
b7_df = []
r7_df = []
res_a_df = []
res_b_df = []
res_c_df = []
zipcodes_df = []



for m in results:
    zipkey = list(m.keys())[0]
    counter = zip_counts[zipkey]
    # All parameters
    b0 = m[zipkey][0].x[0]
    b1 = m[zipkey][0].x[1]
    b2 = m[zipkey][0].x[2]
    # NOTE: COMMENT THIS OUT AFTER TESTING
    r2 = m[zipkey][0].x[3]
    #NOTE: UNCOMMENT THEM, IF TESTING ABOVE DOESN'T WORK
    #r1 = m[zipkey][0].x[3]
    #r2 = m[zipkey][0].x[4]
    b3 = m[zipkey][1].x[0]
    r3 = m[zipkey][1].x[1]
    b4 = m[zipkey][1].x[2]
    r4 = m[zipkey][1].x[3]
    b5 = m[zipkey][2].x[0]
    r5 = m[zipkey][2].x[1]
    b6 = m[zipkey][2].x[2]
    r6 = m[zipkey][2].x[3]
    b7 = m[zipkey][2].x[4]
    r7 = m[zipkey][2].x[5]
    res_a = m[zipkey][0].fun
    res_b = m[zipkey][1].fun
    res_c = m[zipkey][2].fun
    
    b0_df.append([b0]*counter)
    b1_df.append([b1]*counter)
    b2_df.append([b2]*counter)
    #NOTE: UNCOMMENT r1_df AFTER TESTING
    #r1_df.append([r1]*counter)
    r2_df.append([r2]*counter)
    b3_df.append([b3]*counter)
    r3_df.append([r3]*counter)
    b4_df.append([b4]*counter)
    r4_df.append([r4]*counter)
    b5_df.append([b5]*counter)
    r5_df.append([r5]*counter)
    b6_df.append([b6]*counter)
    r6_df.append([r6]*counter)
    b7_df.append([b7]*counter)
    r7_df.append([r7]*counter)
    res_a_df.append(res_a)
    res_b_df.append(res_b)
    res_c_df.append(res_c)
    zipcodes_df.append([zipkey]*counter)
    

b0_df = list(itertools.chain.from_iterable(b0_df))
#print (b0_df)
b1_df = list(itertools.chain.from_iterable(b1_df))
b2_df = list(itertools.chain.from_iterable(b2_df))
# NOTE: UNCOMMENT r1_df table
#r1_df = list(itertools.chain.from_iterable(r1_df))
r2_df = list(itertools.chain.from_iterable(r2_df))
b3_df = list(itertools.chain.from_iterable(b3_df))
r3_df = list(itertools.chain.from_iterable(r3_df))
b4_df = list(itertools.chain.from_iterable(b4_df))
r4_df = list(itertools.chain.from_iterable(r4_df))
b5_df = list(itertools.chain.from_iterable(b5_df))
r5_df = list(itertools.chain.from_iterable(r5_df))
b6_df = list(itertools.chain.from_iterable(b6_df))
r6_df = list(itertools.chain.from_iterable(r6_df))
b7_df = list(itertools.chain.from_iterable(b7_df))
r7_df = list(itertools.chain.from_iterable(r7_df))
res_a_df = list(itertools.chain.from_iterable(res_a_df))
res_b_df = list(itertools.chain.from_iterable(res_b_df))
res_c_df = list(itertools.chain.from_iterable(res_c_df))
zipcodes_df =  list(itertools.chain.from_iterable(zipcodes_df))

In [7]:
# Create dataframe of all 
#NOTE: UNCOMMENT 'r1':r1_df AFTER TESTING
all_outputs_df = pd.DataFrame({'b0':b0_df,'b1':b1_df,'b2':b2_df,#'r1':r1_df,
                              'r2':r2_df,'b3':b3_df,'r3':r3_df,'b4':b4_df,
                              'r4':r4_df,'b5':b5_df,'r5':r5_df,'b6':b6_df,
                              'r6':r6_df,'b7':b7_df,'r7':r7_df,'res_a':res_a_df,
                              'res_b':res_b_df,'res_c':res_c_df,'zip3':zipcodes_df})

In [8]:
# Merge dataframes
final_merged_df = pd.concat([zip3_kadcyla_s1, all_outputs_df], axis=1, join_axes=[zip3_kadcyla_s1.index])


In [10]:
#data model_fits;
#	set DOC.ZIP3_KADCYLA_S3_FINAL;
#	ABS_R = abs(C_R);
#	SQ_C_R = C_R**2;
#	if C_R ne 0 OR KADCYLA_SALES_SA ne 0;
#run; 

final_merged_df['abs_r'] = abs(final_merged_df['res_c'])
final_merged_df['sq_c_r'] = final_merged_df['res_c']**2


In [ ]:
#Proc sql;
#create table DOC.ZIP3_KADCYLA_MAPE as 
#select sum(zip_error)/count(*) as KADCYLA_MAPE from 
#	(select SUM(KADCYLA_SALES_SA) as sales,abs(SUM(C_R)/SUM(KADCYLA_SALES_SA)) AS zip_error from model_fits group by zip3)
#where sales > 0;

zip_mape = final_merged_df.groupby(['ZIP3'])[[SALES_SA,"res_c"]].sum()
zip_mape = zip_mape[zip_mape[SALES_SA] > 0]
mape = sum(list(abs(zip_mape["res_c"]/zip_mape[SALES_SA])))/zip_mape.shape[0]
mape

#SAS MAPE: 0.069644298
#Python MAPE: 208.6133551801872

In [12]:
#Proc sql;
#create table DOC.ZIP3_KADCYLA_R_SQ as 
#select 
#SUM(ABS_R)/SUM(KADCYLA_SALES_SA) AS KADCYLA_FORECAST_ERROR,
#(1-SUM(SQ_C_R)/(VAR(KADCYLA_SALES_SA)*COUNT(*))) AS KADCYLA_R_SQ
#from model_fits;

forecast_error = sum(final_merged_df['abs_r'])/sum(final_merged_df[SALES_SA])
forecast_error

#Forecast error (SAS): 0.223187843
#Forecast error (Python): 0.171

0.17107704682579242

In [16]:
# Adstock transformations

final_merged_df['adstock_calls'] = final_merged_df['b2']* (1 - np.exp(-final_merged_df['r2'] * final_merged_df[CALLS]))
final_merged_df['adstock_sp'] = final_merged_df['b3']* (1 - np.exp(-final_merged_df['r3'] * final_merged_df[SPEAKER_PROG]))
final_merged_df['adstock_copay'] = final_merged_df['b4']* (1 - np.exp(-final_merged_df['r4'] * final_merged_df[COPAY]))
final_merged_df['adstock_emails'] = final_merged_df['b5']* (1 - np.exp(-final_merged_df['r5'] * final_merged_df[EMAILS]))
final_merged_df['adstock_alerts'] = final_merged_df['b6']* (1 - np.exp(-final_merged_df['r6'] * final_merged_df[ALERTS]))
final_merged_df['adstock_webvisits'] = final_merged_df['b7']* (1 - np.exp(-final_merged_df['r7'] * final_merged_df[WEBVISITS]))



#final_merged_df.columns

#b0 + b1*SALES_L1
#b2 * (1 - np.exp(-r2 * CALLS))
#b3 * (1 - np.exp(-r3 * SPEAKER_PROG))
#b4 * (1 - np.exp(-r4 * COPAY))
#b5 * (1 - np.exp(-r5 * EMAILS))
#b6 * (1 - np.exp(-r6 * ALERTS))
#b7 * (1 - np.exp(-r7 * WEBVISITS))



0        2.850820
1        2.988545
2        3.065832
3        3.133543
4        3.159168
5        2.988545
6        3.161582
7        0.000000
8        1.388736
9        0.000000
10       0.061828
11       0.106065
12       0.035063
13       0.112125
14       0.081182
15       0.061828
16       0.055241
17       0.007159
18       0.000000
19       0.000000
20       0.000000
21       0.000000
22       0.000000
23       0.000000
24       0.000000
25       0.000000
26       0.000000
27       8.241798
28      16.115567
29      15.483041
          ...    
7287    -0.207141
7288    -0.000000
7289    -0.000000
7290     0.000000
7291     0.000000
7292     0.000000
7293     0.000000
7294     0.000000
7295     0.000000
7296     0.000000
7297     0.000000
7298     0.000000
7299     0.000000
7300     0.000000
7301     0.000000
7302     0.000000
7303     0.000000
7304     0.000000
7305     0.000000
7306     0.000000
7307     0.000000
7308     0.000000
7309     0.000000
7310     0.000000
7311     0

# Comparison with SAS output

In [53]:
sas_output_model_output = pd.read_csv('../sas_output/MODEL_FITS.csv')
#sas_output_model_output.columns

zip_mape_sas = sas_output_model_output.groupby(['ZIP3'])[["KADCYLA_SALES_SA","C_R"]].sum()
zip_mape_sas = zip_mape_sas[zip_mape_sas['KADCYLA_SALES_SA'] > 0]
sum(list(abs(zip_mape_sas["C_R"]/zip_mape_sas["KADCYLA_SALES_SA"])))/zip_mape_sas.shape[0]


#sas_output = pd.read_csv('../sas_output/ZIP3_KADCYLA_S3_FINAL.csv')

#0.069644298


0.0696442975272081

# Appendix and Development

# MMix model (R Implementation)

In [5]:
%%R -i zip3_kadcyla_s1

sales <- "KADCYLA_SALES_SA"
sales_lag <- "KADCYLA_SALES_SA_L1"
calls <- "KADCYLA_CALLS"
calls_lag <- "KADCYLA_CALLS_L1"

sp <- 'KADCYLA_SP'
copay <- 'KADCYLA_COPAY_COUNT' 
emails <- 'KADCYLA_EMAILS'
alerts <- 'KADCYLA_ALERTS'
webvisits <- 'KADCYLA_WEB_VISITS'


# Formula
formula1 <- as.formula(paste0(sales," ~ b0 + b1*",sales_lag," + 
  (1-exp(-r1*",calls,")) + b2*(1-exp(-r2*",calls_lag,"))"))

formula2 <- as.formula(paste0("residual_a ~ b3*(1 - exp(-r3*",sp,")) + b4*(1 - exp(-r4*",copay,"))"))

formula3 <- as.formula(paste0("residual_b ~ b5*(1 - exp(-r5*",emails,
             ")) + b6*(1 - exp(-r6*",alerts,")) + b7*(1 - exp(-r7*",webvisits,"))"))


#residual_a ~ b3*(1 - exp(-r1*sp)) + b4*(1 - exp(-r1*copay))

#SALES[1] = sqrt(M) + b* D*0 + b*(1 - EXP(-R*CALLS[1]))
model.1 = nlsLM(formula1,
             data = zip3_kadcyla_s1,
             algorithm = "port", control = nls.lm.control(maxfev = 1000, maxiter = 1000),
             start     = c(b0 = 1, b1=1, r1=0.01,r2=0.01,b2 = 0),
             lower     = c(b0 = -Inf, b1=-Inf, r1=-Inf,r2=-Inf,b2 = -Inf),
             upper     = c(b0 = Inf, b1=Inf, r1=Inf,r2=Inf,b2 = Inf)
    )

# Prediction from first model
residual_a <- zip3_kadcyla_s1[,sales] - predict(model.1)

model.2 = nlsLM(formula2,
             data = zip3_kadcyla_s1,
             algorithm = "port", control = nls.lm.control(maxfev = 1000, maxiter = 1000),
             start     = c(b3 = 1, b4=1, r3=0.01,r4=0.01),
             lower     = c(b3 = -Inf, b4=-Inf, r3=-Inf,r4=-Inf),
             upper     = c(b3 = Inf, b4=Inf, r3=Inf,r4=Inf)
    )

residual_b <- residual_a - predict(model.2)
model.3 = nlsLM(formula3,
             data = zip3_kadcyla_s1,
             algorithm = "port", control = nls.lm.control(maxfev = 1000, maxiter = 1000),
             start     = c(b5 = 1, b6=1, b7=1, r5=0.01,r6=0.01,r7=0.01),
             lower     = c(b5 = -Inf, b6=-Inf, b7=-Inf, r5=-Inf,r6=-Inf,r7=-Inf),
             upper     = c(b5 = Inf, b6=Inf, b7=Inf, r5=Inf,r6=Inf,r7=Inf)
    )

residual_c <- residual_b - predict(model.3)

param_ests <- c(summary(model.1)$parameters[,"Estimate"],
summary(model.2)$parameters[,"Estimate"],
summary(model.3)$parameters[,"Estimate"])

#param_ests

for (zip in unique(zip3_kadcyla_s1[,"ZIP3"])){
    datasub <- zip3_kadcyla_s1[zip3_kadcyla_s1[,"ZIP3"] == zip,]
    model.1.sub = nlsLM(formula1,
             data = datasub,
             algorithm = "port", control = nls.lm.control(maxfev = 1000, maxiter = 1000),
             start     = c(b0 = 0, b1=0, r1=0.01,r2=0.01,b2 = 0),
             lower     = c(b0 = 0, b1=0, r1=-Inf,r2=-Inf,b2 = 0),
             upper     = c(b0 = Inf, b1=Inf, r1=Inf,r2=Inf,b2 = Inf)
    )
    print(summary(model.1.sub)$parameters[,"Estimate"])
}


#residual_b
#summary(modcalls)


Error in nlsModel(formula, mf, start, wts) : 
  singular gradient matrix at initial parameter estimates


/Users/faruqia/anaconda3/envs/ppm-r-env/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in nlsModel(formula, mf, start, wts) : 
  singular gradient matrix at initial parameter estimates

  warnings.warn(x, RRuntimeWarning)


In [6]:
#zip3_kadcyla_s1.columns

# Ensemble modeling

In [7]:
X = zip3_kadcyla_s1[['KADCYLA_CALLS', 'KADCYLA_SP', 'KADCYLA_COPAY_COUNT', 'KADCYLA_EMAILS',
'KADCYLA_ALERTS', 'KADCYLA_WEB_VISITS','KADCYLA_CALLS_L1','KADCYLA_SP_L1']]

Y = zip3_kadcyla_s1['KADCYLA_SALES_SA']

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                        test_size=0.33,
                                                        random_state=42)

In [8]:
model1 = RandomForestRegressor(n_estimators=7000,random_state=0,
                                   criterion='mse',min_samples_split=45)

model2 = GradientBoostingRegressor(n_estimators=5000, learning_rate=1,
max_depth=30, random_state=0, min_samples_split=3)

model1.fit(X_train, y_train.values.ravel())
preds_mod1_y1 = model1.predict(X_test)

model2.fit(X_train, y_train.values.ravel())
preds_mod2_y1 = model2.predict(X_test)

In [12]:
# R^2 (coefficient of determination) regression score function. Best score is 1.0
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score

r2_score(y_test,preds_mod1_y1)

0.5564960111113402

In [13]:
r2_score(y_test,preds_mod2_y1)

0.1513579905513891

In [74]:
#data model_fits;
#	set DOC.ZIP3_KADCYLA_S3_FINAL;
#	ABS_R = abs(C_R);
#	SQ_C_R = C_R**2;
#	if C_R ne 0 OR KADCYLA_SALES_SA ne 0;
#run; 



#Proc sql;
#create table DOC.ZIP3_KADCYLA_R_SQ as 
#select 
#SUM(ABS_R)/SUM(KADCYLA_SALES_SA) AS KADCYLA_FORECAST_ERROR,
#(1-SUM(SQ_C_R)/(VAR(KADCYLA_SALES_SA)*COUNT(*))) AS KADCYLA_R_SQ
#from model_fits;

In [ ]:
#sas_output['T0'][9] 
#all_outputs_df['b0'][9]

In [ ]:
# Compare non-zero zip3 of SAS and Python


#MAPE,
#Forecasting error

In [19]:
#T0	T1	T2	A	A_R	T3	T4	B	B_R	T5	T6	T7	C	C_R
#b0,b1,b2,r1,r2, b3,r3,b4,r4, b5,r5,b6,r6,b7,r7



In [ ]:
#sysctl -n hw.ncpu
#4 cpus